### Plotly Dashboard Sample Code Example

In [25]:
#import the packages
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from pylab import plot, show
%matplotlib inline

In [26]:
#Read the csv file
app = dash.Dash()
indicators_df = pd.read_csv('../data/indicators.csv')

In [27]:
#Getting the unique value of indicators
indicators_list = indicators_df['Indicator Name'].unique()
indicators_list

array(['Agriculture, value added (% of GDP)',
       'CO2 emissions (metric tons per capita)',
       'Domestic credit provided by financial sector (% of GDP)',
       'Electric power consumption (kWh per capita)',
       'Energy use (kg of oil equivalent per capita)',
       'Exports of goods and services (% of GDP)',
       'Fertility rate, total (births per woman)',
       'GDP growth (annual %)',
       'GNI per capita, PPP (current international $)',
       'Gross enrollment ratio, primary, both sexes (%)',
       'Gross enrolment ratio, secondary, both sexes (%)',
       'High-technology exports (% of manufactured exports)',
       'Imports of goods and services (% of GDP)',
       'Income share held by lowest 20%',
       'Industry, value added (% of GDP)',
       'Inflation, GDP deflator (annual %)',
       'Internet users (per 100 people)',
       'Life expectancy at birth, total (years)',
       'Military expenditure (% of GDP)',
       'Mobile cellular subscriptions (per 100

In [28]:
#Getting information about indicators
indicators_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85536 entries, 0 to 85535
Data columns (total 5 columns):
Unnamed: 0        85536 non-null int64
Country Name      85536 non-null object
Indicator Name    85536 non-null object
Year              85536 non-null int64
Value             36616 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 3.3+ MB


In [29]:
#check for null value
indicators_df.isnull().sum()

Unnamed: 0            0
Country Name          0
Indicator Name        0
Year                  0
Value             48920
dtype: int64

In [30]:
#Getting the column names
indicators_df.columns

Index(['Unnamed: 0', 'Country Name', 'Indicator Name', 'Year', 'Value'], dtype='object')

In [31]:
#Getting top rows
indicators_df.head()

,Unnamed: 0,Country Name,Indicator Name,Year,Value
0,0,Arab World,"Agriculture, value added (% of GDP)",1952,NaN
1,1,Arab World,CO2 emissions (metric tons per capita),1952,NaN
2,2,Arab World,Domestic credit provided by financial sector (...,1952,NaN
3,3,Arab World,Electric power consumption (kWh per capita),1952,NaN
4,4,Arab World,Energy use (kg of oil equivalent per capita),1952,NaN


In [39]:
app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in indicators_list],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in indicators_list],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=indicators_df['Year'].min(),
        max=indicators_df['Year'].max(),
        value=indicators_df['Year'].max(),
        step=None,
        marks={str(year): str(year) for year in indicators_df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

In [40]:
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = indicators_df[indicators_df['Year'] == year_value]

    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

In [41]:
def create_time_series(dff, axis_type, title):
    return {
        'data': [go.Scatter(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

In [42]:
@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = indicators_df[indicators_df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [43]:
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

In [44]:
if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2018 14:08:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:03] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2018 14:09:34] "POST /_dash-update-component HTTP/1.1" 200 -
